In [1]:
import sys
from pathlib import Path

src_path = (Path("../src")).resolve()

if str(src_path) not in {str(Path(p).resolve()) for p in sys.path if p}:
    sys.path.insert(0, str(src_path))

In [2]:
from market_decision_engine.contract.schemas.policy import (
    UserPolicySnapshot,
    AccountPolicy,
    RiskPolicy,
    ExecutionPolicy,
    MarketConstraints,
    TradePlanPolicy,
    Market,
    Currency,
)
from market_decision_engine.domain.models import ChartLogic

policy = UserPolicySnapshot(
    account=AccountPolicy(
        equity=100_000.0,        # 口座残高（USD想定）
        currency=Currency.USD,
    ),
    risk=RiskPolicy(
        risk_per_trade_pct=0.005,  # 1トレード0.5%
        max_position_pct=0.10,     # 1銘柄10%まで
        max_concurrent_positions=10,
    ),
    execution=ExecutionPolicy(
        slippage_pct=0.001,        # 0.1% スリッページ想定
        commission_per_order=0.0,  # 確認用なのでゼロ
        tax_pct=0.0,
    ),
    constraints=MarketConstraints(
        market=Market.US,
        lot_size=1,                # 米国株
        min_price=5.0,             # 低位株除外
        min_avg_dollar_volume=0.0, # 今は流動性制限なし
        impact_cap_pct=0.0,
    ),
    trade_plan=TradePlanPolicy(
        entry_buffer_pct=0.001,    # 高値ブレイク+0.1%
        atr_n=14,
        atr_stop_k=2.0,
        swing_lookback=20,
        time_stop_days=40,
    ),
)

chart_logic = ChartLogic(policy=policy)
chart_logic.load_charts(["AAPL", "MSFT"])

In [3]:
from pprint import pprint
pprint(chart_logic.plan)

{'account': {'currency': <Currency.USD: 'USD'>, 'equity': 100000.0},
 'constraints': {'impact_cap_pct': 0.0,
                 'lot_size': 1,
                 'market': <Market.US: 'US'>,
                 'min_avg_dollar_volume': 0.0,
                 'min_price': 5.0},
 'execution': {'commission_per_order': 0.0,
               'slippage_pct': 0.001,
               'tax_pct': 0.0},
 'risk': {'max_concurrent_positions': 10,
          'max_position_pct': 0.1,
          'risk_per_trade_pct': 0.005},
 'trade_plan': {'atr_n': 14,
                'atr_stop_k': 2.0,
                'entry_buffer_pct': 0.001,
                'swing_lookback': 20,
                'time_stop_days': 40}}


In [4]:
scnaned = chart_logic.scan(ticker="AAPL", view="ohlcv")  # AAPLのOHLCVデータを表示
print(scnaned)

scnaned = chart_logic.scan(ticker="AAPL", view="indicators")  # AAPLのOHLCVデータを表示
print(scnaned)

scnaned = chart_logic.scan(ticker="AAPL", view="decision")  # AAPLのOHLCVデータを表示
print(scnaned)

| date       |       open |       high |        low |      close |           volume |
|------------|------------|------------|------------|------------|------------------|
| 2025-12-18 | 273.609985 | 273.630005 | 266.950012 | 272.190002 |  51630700.000000 |
| 2025-12-19 | 272.149994 | 274.600006 | 269.899994 | 273.670013 | 144632000.000000 |
| 2025-12-22 | 272.859985 | 273.880005 | 270.510010 | 270.970001 |  36571800.000000 |
| 2025-12-23 | 270.839996 | 272.500000 | 269.559998 | 272.359985 |  29642000.000000 |
| 2025-12-24 | 272.339996 | 275.429993 | 272.200012 | 273.809998 |  17910600.000000 |
| 2025-12-26 | 274.160004 | 275.369995 | 272.859985 | 273.399994 |  21521800.000000 |
| 2025-12-29 | 272.690002 | 274.359985 | 272.350006 | 273.760010 |  23715200.000000 |
| 2025-12-30 | 272.809998 | 274.079987 | 272.279999 | 273.079987 |  22139600.000000 |
| 2025-12-31 | 273.059998 | 273.679993 | 271.750000 | 271.859985 |  27293600.000000 |
| 2026-01-02 | 272.260010 | 277.839996 | 269.000000 | 